# Surprise SVD vs SAR

NOTE: whenever there is ..., it means that we need to add more information.

This notebook showcase Surprise SVD vs SAR ...

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import surprise
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from utilities.common.file_utils import maybe_download
from utilities.recommenders.surprise.utils import surprise_predictions_to_numpy
from utilities.datasets.splitter import RandomSplitter

print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Pandas: ", pd.__version__)
print("Surprise: ", surprise.__version__)

%load_ext autoreload
%autoreload 2

OS:  darwin
Python:  3.6.0 | packaged by conda-forge | (default, Feb 10 2017, 07:08:35) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]
Numpy:  1.14.3
Pandas:  0.23.3
Surprise:  1.0.6
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# Parameters
TEST_SIZE = 0.25
EPOCHS = 10
SVD_FACTORS = 100

## Dataset
The dataset we are going to use is MovieRatings, which consists of ...

In [12]:
dataset_url = 'http://aka.ms/MovieRatings.csv'
filename = maybe_download('MovieRatings.csv', dataset_url, verbose=True)

File MovieRatings.csv already downloaded


In [13]:
df = pd.read_csv(filename)
df.head()

,UserId,MovieId,Rating,Timestamp
0,1,68646,10,1381620027
1,1,113277,10,1379466669
2,2,454876,8,1394818630
3,2,790636,7,1389963947
4,2,816711,8,1379963769


## Surprise SVD
Surprise is an open source library that contains ...

Singular Value Decomposition (SVD) is a form of matrix factorization in which the columnsofUandVare constrained to be mutually orthogonal ...

### Prepare data for Surprise


In [32]:
#reader = Reader(line_format='user item rating timestamp', rating_scale=(df['Rating'].min(), df['Rating'].max()))
#data = Dataset.load_from_df(df, reader)
#data = Dataset.load_from_df(df_surprise, reader)

reader = Reader(rating_scale=(df['Rating'].min(), df['Rating'].max()))
df_surprise = df.drop('Timestamp', axis=1) 
data = Dataset.load_from_df(df_surprise, reader)


In [36]:
#TODO: use our own splitter
splitter = RandomSplitter(ratio=1 - TEST_SIZE)
trainset, testset = splitter.split(data)


### Modeling

In [43]:
algo = SVD(n_factors=SVD_FACTORS, n_epochs=EPOCHS, verbose=True)
algo.fit(trainset)
predictions = algo.test(testset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


### Evaluation


In [44]:
y_true, y_pred = surprise_predictions_to_numpy(predictions)
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
print("RMSE={}".format(np.sqrt(mse)))
print("MAE={}".format(mae))

RMSE=1.5400390625
MAE=1.1328125


/Users/miguel/anaconda3/envs/airship/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


# SAR
Smart Adaptative Recommendation (SAR) is an algorithm ...